In [ ]:
import tensorflow as tf

# !pip install -q tensorflow-datasets

In [ ]:
 
import tensorflow_datasets as tfds
import numpy as np


# **Data processing**

In [ ]:
#We use tensorflow_datasets to load dataset with the method load()
#load() needs the name of the dataset
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())




In [ ]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
vocab_size = 10000 #the maximum number of words to keep in Tokenizer
embedding_dim = 16 
max_length = 120 #Maximum length of our sequences
trunc_type='post' #remove values from sequences larger than max_length, at the end of the sequences. 
oov_tok = "<OOV>" #Out of vocabulary token
num_epochs = 20 #Number of epochs execute without the earlystopping
BATCH_SIZE = 500 


**Preparation of data**

In [ ]:
#Tokenization and padding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#We use the Tokenizer class 

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

#Transforms each text in texts to a sequence of integers.
sequences = tokenizer.texts_to_sequences(training_sentences)
#We pad sequences to the same length using pad_sequences function
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [ ]:
#Definition of callbacks
#To stop  the training when the metric dont change to avoid overfitting
stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

#Save the weights for the best(max) metric value
#for GRU
checkpoint_gru_filepath = '/tmp/checkpoint_gru'
model_checkpoint_gru_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_gru_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#for LSTM
checkpoint_lstm_filepath = '/tmp/checkpoint_lstm'
model_checkpoint_lstm_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_lstm_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#for CNN
checkpoint_cnn_filepath = '/tmp/checkpoint_cnn'
model_checkpoint_cnn_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_cnn_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

**GRU Model**

In [ ]:
# Model Definition with GRU
model_gru = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_gru.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_gru.summary()


Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
dropout_46 (Dropout)         (None, 120, 16)           0         
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 32)                3264      
_________________________________________________________________
dense_48 (Dense)             (None, 6)                 198       
_________________________________________________________________
dropout_47 (Dropout)         (None, 6)                 0         
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 7         
Total params: 163,469
Trainable params: 163,469
Non-trainable params: 0
_______________________________________________

In [ ]:
#trainning with the specified batch size and the callbacks
model_gru.fit(padded, training_labels_final, epochs=num_epochs, validation_split=0.25, callbacks = [stopping_callback, model_checkpoint_gru_callback], batch_size= BATCH_SIZE)

Epoch 1/20
38/38 [==============================] - 3s 80ms/step - loss: 0.6931 - accuracy: 0.5041 - val_loss: 0.6926 - val_accuracy: 0.5282
Epoch 2/20
38/38 [==============================] - 2s 63ms/step - loss: 0.6914 - accuracy: 0.5230 - val_loss: 0.6903 - val_accuracy: 0.5467
Epoch 3/20
38/38 [==============================] - 2s 62ms/step - loss: 0.6823 - accuracy: 0.5708 - val_loss: 0.6731 - val_accuracy: 0.6150
Epoch 4/20
38/38 [==============================] - 2s 62ms/step - loss: 0.5817 - accuracy: 0.6991 - val_loss: 0.4623 - val_accuracy: 0.8082
Epoch 5/20
38/38 [==============================] - 2s 62ms/step - loss: 0.4413 - accuracy: 0.8009 - val_loss: 0.4074 - val_accuracy: 0.8229
Epoch 6/20
38/38 [==============================] - 2s 62ms/step - loss: 0.3892 - accuracy: 0.8320 - val_loss: 0.3886 - val_accuracy: 0.8315
Epoch 7/20
38/38 [==============================] - 2s 63ms/step - loss: 0.3470 - accuracy: 0.8550 - val_loss: 0.3953 - val_accuracy: 0.8325
Epoch 8/20
38

In [ ]:
#Load the saved best weights and evaluate the model
model_gru.load_weights(checkpoint_gru_filepath)
evaluation = model_gru.evaluate(testing_padded,testing_labels_final)


782/782 [==============================] - 13s 17ms/step - loss: 0.3993 - accuracy: 0.8350


**LSTM Model**

In [ ]:
# Model Definition with LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_lstm.summary()


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
dropout_48 (Dropout)         (None, 120, 16)           0         
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 64)                12544     
_________________________________________________________________
dense_50 (Dense)             (None, 6)                 390       
_________________________________________________________________
dropout_49 (Dropout)         (None, 6)                 0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 7         
Total params: 172,941
Trainable params: 172,941
Non-trainable params: 0
_______________________________________________

In [ ]:
#training
model_lstm.fit(padded, training_labels_final, epochs=num_epochs, validation_split=0.25, callbacks = [stopping_callback, model_checkpoint_lstm_callback], batch_size=BATCH_SIZE)

Epoch 1/20
38/38 [==============================] - 3s 85ms/step - loss: 0.6909 - accuracy: 0.5299 - val_loss: 0.6790 - val_accuracy: 0.6363
Epoch 2/20
38/38 [==============================] - 2s 65ms/step - loss: 0.5994 - accuracy: 0.7091 - val_loss: 0.4844 - val_accuracy: 0.7997
Epoch 3/20
38/38 [==============================] - 2s 65ms/step - loss: 0.4775 - accuracy: 0.8202 - val_loss: 0.4461 - val_accuracy: 0.8142
Epoch 4/20
38/38 [==============================] - 2s 65ms/step - loss: 0.4149 - accuracy: 0.8533 - val_loss: 0.4127 - val_accuracy: 0.8413
Epoch 5/20
38/38 [==============================] - 2s 64ms/step - loss: 0.3816 - accuracy: 0.8710 - val_loss: 0.4018 - val_accuracy: 0.8333
Epoch 6/20
38/38 [==============================] - 2s 64ms/step - loss: 0.3638 - accuracy: 0.8828 - val_loss: 0.4305 - val_accuracy: 0.8389
Epoch 7/20
38/38 [==============================] - 2s 64ms/step - loss: 0.3368 - accuracy: 0.8967 - val_loss: 0.4108 - val_accuracy: 0.8262


In [ ]:
#Load the best weights and evaluate the model
model_lstm.load_weights(checkpoint_lstm_filepath)
evaluation = model_lstm.evaluate(testing_padded,testing_labels_final)


782/782 [==============================] - 14s 17ms/step - loss: 0.4086 - accuracy: 0.8408


**CNN Model**

In [ ]:
# Model Definition with Conv1D
model_cnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_cnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_cnn.summary()


Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 116, 128)          10368     
_________________________________________________________________
global_average_pooling1d_5 ( (None, 128)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 6)                 774       
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 7         
Total params: 171,149
Trainable params: 171,149
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training
model_cnn.fit(padded, training_labels_final, epochs=num_epochs, validation_split= 0.25, callbacks = [stopping_callback, model_checkpoint_cnn_callback], batch_size=BATCH_SIZE)

Epoch 1/20
38/38 [==============================] - 1s 28ms/step - loss: 0.6892 - accuracy: 0.5568 - val_loss: 0.6741 - val_accuracy: 0.6578
Epoch 2/20
38/38 [==============================] - 1s 26ms/step - loss: 0.6077 - accuracy: 0.7446 - val_loss: 0.5172 - val_accuracy: 0.7998
Epoch 3/20
38/38 [==============================] - 1s 25ms/step - loss: 0.4068 - accuracy: 0.8431 - val_loss: 0.3951 - val_accuracy: 0.8221
Epoch 4/20
38/38 [==============================] - 1s 26ms/step - loss: 0.2969 - accuracy: 0.8800 - val_loss: 0.3636 - val_accuracy: 0.8406
Epoch 5/20
38/38 [==============================] - 1s 25ms/step - loss: 0.2444 - accuracy: 0.9052 - val_loss: 0.3650 - val_accuracy: 0.8427
Epoch 6/20
38/38 [==============================] - 1s 23ms/step - loss: 0.2060 - accuracy: 0.9245 - val_loss: 0.3755 - val_accuracy: 0.8442
Epoch 7/20
38/38 [==============================] - 1s 25ms/step - loss: 0.1781 - accuracy: 0.9381 - val_loss: 0.3993 - val_accuracy: 0.8397
Epoch 8/20
38

In [ ]:
#Load the best weights and evaluate the model
model_cnn.load_weights(checkpoint_cnn_filepath)
evaluation = model_cnn.evaluate(testing_padded,testing_labels_final)


782/782 [==============================] - 3s 3ms/step - loss: 0.3729 - accuracy: 0.8425
